# Power Length V5

This takes a phase diagram - varies cavity length and power, and takes spectral readings, images and output power at each point. 

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

import time
import logging
import winsound
import numpy as np
from tqdm.notebook import tqdm
import Components as comp
from Measure import power_scan
import socket
import sys
if socket.gethostname() == "ph-photonbec5":
    sys.path.append(r"D:/Control/PythonPackages/")

from pbec_analysis import make_timestamp

C:\Semiconductor Cavity\WheelBEC\Experimental Code\Measure.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Initialise Equipment
If equipment can take measurements, can set measure = True or False when initialising them. Note some components are interdependent.

In [2]:
components = dict()

components.update({"laser": comp.Toptica_Laser(com_port='COM12')})
components.update({"powermeter": comp.Thor_PowerMeter(power_meter_usb_name='USB0::0x1313::0x8078::P0034379::INSTR', num_power_readings=100, bs_factor=0.5, wavelength=950)})
components.update({"spectrometer": comp.Spectrometer(spec_nd=1 / 30.5)})
# components.update({"camera": comp.FLIR_Camera(measure=True, )})
components.update({"wheel": comp.FilterWheel(com_port='COM9')})

### Choose loop parameters

In [3]:
# components['laser'].set(248e-3)

In [4]:

#Define PCA range
length_list = np.linspace(8, -1, 7)
stage_list = np.linspace(0, 4e6, 8, dtype=int)
stage_list = stage_list + 0.0
#Define power range
power_list = np.linspace(15e-3, 245e-3, 20)

type(stage_list[0])

numpy.float64

### Measurement Loop
Note Kinesis stage needs to be closed each time it is used, or won't be able to open again - so only use the stage using 'with' statements. 

In [ ]:
# %qtconsole

time_stamps = []
logging.basicConfig(filename = fr'Logs\power_lengthV5_{make_timestamp()}.log', filemode='w', level=logging.INFO, force=True)
print('hi')
with comp.Translation_Stage(26000913, is_rack_system = False, scale = 1) as stage, comp.FLIR_Camera(measure=True, algorithm='rising') as camera:
    components.update({"camera": camera})
    pbar = tqdm(stage_list)
    for s in pbar:
        stage.set(s)
        pbar.set_description(f'Test Displacement {s}')
        time.sleep(1)
    
    for s in tqdm(stage_list):
        stage.set(s)
        time.sleep(1)
        comp.params.update({"stage_pos": s})
        
        #Slowly return to initial wavelength
        for l in np.flip(length_list):
            comp.set_lock(l)
            time.sleep(5)
        
        for l in tqdm(length_list, leave=True):
            comp.set_lock(l) #save pca value
            ts = power_scan(power_list, components, l)
            time_stamps.append(ts)
    stage.set(0)
    components['laser'].set(0)
print("'"+str(time_stamps[0][0])+"'"+","+"'"+str(time_stamps[-1][-1])+"'")
winsound.Beep(1000, 2000)



hi
Here are the camera names:
{'blackfly_minisetup': '19128822', 'blackfly_semiconductor_cavity': '17458446', 'blackfly_semiconductor_cavity_lock': '19441065', 'bonus_chameleon': '14110699', 'nathans_dungeon_lock_NA0': '17458446', 'nathans_dungeon_cavity_NA1': '19128822'}


-> Detected 2 USB3 cameras


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]